# HELOC Data Scientist

## Primary Objective
Provide graphical and summary explanations of three types of directly interpretable (glassbox) data-inductive (Machine Learning) models for credit risk assessment.  The dataset under investigation is the FICO Home Equity Loan data (HELOC).  This was compiled from real-world financial data and was used in the FICO Explainable ML Models Challenge. 

The models classify new instances as `nondefault` or `default`.  The components of the model need to be accessible in order to provide interpretations of model operation and predictions.  

There are four model types in the `interpretML` package.  The Decision Tree Classifier was not selected due to the potential for overfitting since the HELOC dataset contains 34 features with approximately 7,000 instances in the training set.

The end goal is to create a dashboard directed towards the requirements of a Data Scientist / Model Developer.

###### Contents
1. [Feature Importance](#feature-importance)

2. [Model Simplification](#model-simple)

3. [Evaluation and Presentation](#eval-present)

4. [Feature Selection](#select)
---

Set a seed for reproducibility, and import packages.

In [1]:
import numpy as np
import pandas as pd
import os.path
import urllib.request
import pprint

from pandas import read_csv
from matplotlib import pyplot

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn import datasets, model_selection, ensemble, metrics, pipeline, preprocessing

from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from interpret.glassbox import LogisticRegression
from interpret import show

seed = 2022

---

## 1. Global Feature Importance <a name="feature-importance"></a>
Source data is the Home Equity Loan [HELOC](https://community.fico.com/s/explainable-machine-learning-challenge?tabset-158d9=3) data set (as also used in the FICO Explainability Challenge). 

1. Load the data set (and print its characteristics)  
2. Train directly interpretable (glassbox) models on the data
3. Identify Global Feature Importance

In [2]:
url = 'https://raw.githubusercontent.com/JJCoen/Interpretable-ML-Models/master/data/train_imp.csv'
train = pd.read_csv(url)
del train['Unnamed: 0']

url2 = 'https://raw.githubusercontent.com/JJCoen/Interpretable-ML-Models/master/data/test_imp.csv'
test = pd.read_csv(url2)
del test['Unnamed: 0']

In [3]:
print( train.shape)
print( test.shape)
features = train.columns[1:]
label = train.columns[0]

(7403, 35)
(2468, 35)


Encode a positive classification (1) as being a default.  
A negative classification (0) is a nondefault.  In $R$, this is the reference level

In [4]:
X_train = train[features]
y_train = train[label].apply(lambda x: 0 if x == "Good" else 1) #Turning response into 0 and 1
print(X_train.shape)
label_names = train[label].unique()
label_names

(7403, 34)


array(['Bad', 'Good'], dtype=object)

In [5]:
X_test = test[features]
y_test = test[label].apply(lambda x: 0 if x == "Good" else 1) #Turning response into 0 and 1
print(X_test.shape)
print(y_test.head())

(2468, 34)
0    1
1    1
2    0
3    1
4    1
Name: RiskPerformance, dtype: int64


In [6]:
X_train.head().transpose()

,0,1,2,3,4
ExternalRiskEstimate,70.000000,62.00,68.000,66.000000,74.00000
MSinceOldestTradeOpen,145.000000,254.00,222.000,95.000000,191.00000
MSinceMostRecentTradeOpen,1.000000,20.00,2.000,21.000000,4.00000
AverageMInFile,11.000000,69.00,93.000,64.000000,66.00000
NumSatisfactoryTrades,8.000000,11.00,32.000,18.000000,37.00000
NumTrades60Ever2DerogPubRec,0.000000,5.00,0.000,1.000000,0.00000
NumTrades90Ever2DerogPubRec,0.000000,4.00,0.000,1.000000,0.00000
PercentTradesNeverDelq,100.000000,69.00,97.000,100.000000,97.00000
MSinceMostRecentDelq,32.788333,24.00,16.000,33.646667,13.00000
MaxDelq2PublicRecLast12M,7.000000,6.00,6.000,6.000000,6.00000


In [7]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')

#### Train Models
Cohort models can be on the complete training set, test set, or a group of applicants that a Regulator may want to inspect.

In [8]:
ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

lr = LogisticRegression(random_state=seed, feature_names=features, penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)

#### Cohort-wide Explanations: What the model learned overall
**Explainable Boosting Machine**

In [9]:
ebm_global = ebm.explain_global(name='EBM-Loan')
#show(ebm_global)
show(ebm_global, key = 1)

C:\Users\jimco\anaconda3\lib\site-packages\interpret\visual\udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\jimco\anaconda3\lib\site-packages\interpret\visual\udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\jimco\anaconda3\lib\site-packages\interpret\visual\udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7001/2612967364544/ -->

#### Overall Importance
The EBM measures feature importance in terms of mean absolute score.  

In the Generalised Additive Model, the response variable, $Y$, is given by the addition of smooth functions acting on each predictor variable:  
$$
g(E[Y]) = \beta_0 + f_1(x_1) + f_2(x_2) + \quad . . . 
$$

$Y$ takes on values of 0 or 1 so $E[Y]$ is a binomial and the link function, $g$, is the logit or the $\text{log(Odds)}$. 

For a single instance, the function $f_i$ acts on the feature value ($x_i$).  This contributes to the $\text{log(Odds)}$ for that instance.  The mean absolute score for a feature is the average of the contributions over all instances.  In addition, this value is weighted by the density in the training set.  So, if 90% of the instances had a classification of "0", then these instances have a weighting of 0.9.  The other instances with a "1" classification have a weighting of 0.1.  This is to prevent extreme values from having undue influence.


**Logistic Regression**

In [10]:
lr_global = lr.explain_global(name='LR-Loan')
show(lr_global)

<!-- http://127.0.0.1:7001/2613483678064/ -->

A high positive intercept dominates over the coefficient values.  It is possible to move the cursor over the plot so as to select the range of the coefficients

#### Case-specific Feature Importance

Select specific instances by selecting from the Component menu

In [11]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5])
show(ebm_local)

<!-- http://127.0.0.1:7001/2613483933504/ -->

In [12]:
lr_local = lr.explain_local(X_test[:5], y_test[:5])
show(lr_local)

<!-- http://127.0.0.1:7001/2613483967968/ -->

---

## Model Simplification <a name="model-simple"></a>
The `DecisionListClassifier` gives a set of logical rules for making a positive prediction.  They are ranked according to precision:

> Precision is the ratio between the True Positives and   
> the predicted positives. 
 
$$ 
\begin{aligned}
\text{Precision} &= \frac{TP}{\text{predicted positive}} \\
&= \frac{TP}{TP + FP}
\end{aligned}
$$

`Precision` gives a measure of the instances that are relevant. It is used when it is necessary to minimise the number of false positives.  In credit risk, this means reducing the number of applicants predicted as `default` when they are more likely to repay (`nondefault`).

A secondary measure is `Recall`.  

> Recall is the ratio of the true positives to all the condition positives

Recall gives a measure of the model correctly identifying True Positives. It gives the proportion of instances that the model identified correctly.  It is the ratio of the true positives to all the condition positives.  
Mathematically:  
$$ Recall = \frac{TP}{TP + FN}$$

For a binary classifier, this is the __Sensitivity__ or True Positive Rate. This is an important metric.  

In situations where it is critical that a class is identified correctly, as in medicine, the classifier needs to have high `recall`. 

`Precision` aims to minimise false positives.  If we need to avoid classifying patients as _covid_ positive when they disease-free,  we should aim for high precision.  A high Precision value is a way to ensure that resources go to those patients that need them and not to the _false positives_. 

Ideally, the model should have high precision and high recall values.  However, often it is not possible to achieve both goals at the same time.  Rules are ranked according to precision.

In [13]:
seed = 2022
dl = DecisionListClassifier(random_state=seed)
dl.fit(X_train, y_train)

dl_global = dl.explain_global(name='DL-Loan')

dl_local = dl.explain_local(X_test[:5], y_test[:5])

show(dl_global)

C:\Users\jimco\anaconda3\lib\site-packages\sklearn\externals\six.py:28: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



<!-- http://127.0.0.1:7001/2612028229760/ -->

---

## Evaluation and Presentation <a name="eval-present"></a>
* Compute the F1 score for the EBM model and   
* Calculate AUC scores for three glassbox models and present as a dashboard

In [14]:
from interpret.perf import ROC
# Print out the classifier performance (F1-score)
print('Classifier performance (F1):', metrics.f1_score(y_test, ebm.predict(X_test), average='weighted'))
f1_34 = metrics.f1_score(y_test, ebm.predict(X_test), average='weighted')

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM-Perf')

lr_perf = ROC(lr.predict_proba).explain_perf(X_test, y_test, name='LR-Perf')

dl_perf = ROC(dl.predict_proba).explain_perf(X_test, y_test, name='DL-Perf')


Classifier performance (F1): 0.7398944621245921


In [15]:
print("Train Accuracy : %.2f" %ebm.score(X_train, y_train) )
print("Test Accuracy : %.2f" %ebm.score(X_test, y_test) )
acc_34 = ebm.score(X_test, y_test)

Train Accuracy : 0.76
Test Accuracy : 0.74


### Dashboard

In [16]:
show([hist, lr_global, lr_perf, dl_global, dl_perf, ebm_global, ebm_perf], share_tables=True)

<!-- http://127.0.0.1:7001/2612030759552/ -->
 Open in new window

---

## Feature Selection <a name="select"></a>

An underlying objective of this research study is to place greater emphasise on explaining ML models to the service user / loan applicant.  To this end, when an application is denyed, the service user wants to know what measures they might undertake in order to obtain approval.  The Constastive Explanations package can identify such correcitve measures.  However, it is capable of working with only ten or fewer features.  In order to facilitate the work of the Loan Officer in providing explanations to the Service User, it is necessary to restrict the EBM model to the ten features with greatest global importance.

In [17]:
# Convert feature importance dict to a dataframe
feat_importance = pd.DataFrame( ebm_global.data() )
feat_importance.columns

Index(['type', 'names', 'scores'], dtype='object')

In [18]:
# Sort by scores
feat_scores = feat_importance.sort_values(['scores'], ascending = False).head(10) 
feat_select = feat_scores['names']

The `top-10.csv` file has already been created.  Un-comment the cell below if you wish to replicate.

In [19]:
#from pathlib import Path  
#filepath = Path('../data/top-10.csv')
#feat_select.to_csv(filepath)

In [19]:
# Recalibrate training and test sets
X_train = train[feat_select]
X_test = test[feat_select]

In [20]:
print(X_train.shape)
print(X_test.shape)

(7403, 10)
(2468, 10)


In [21]:
# Train EBM on reduced training set
ebm_r = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm_r.fit(X_train, y_train)
ebm_select = ebm_r.explain_global(name='EBM-Loan-select')
show(ebm_select)

<!-- http://127.0.0.1:7001/2612028122592/ -->

It is interesting that when the number of features is reduced to the top ten, then the EBM Generalised Additive Model takes pair-wise feature interactions into account.

In [22]:
# Print out the classifier performance (F1-score)
print('Classifier performance (F1): %.3f' %metrics.f1_score(y_test, ebm_r.predict(X_test), average='weighted'))
f1_10 = metrics.f1_score(y_test, ebm_r.predict(X_test), average='weighted')
acc_10 =  ebm_r.score(X_test, y_test)

Classifier performance (F1): 0.736


In [23]:
eval = pd.DataFrame()
eval['EBM model'] = ['full', 'reduced']
eval['f1'] = [f1_34, f1_10]
eval['acc'] = [acc_34, acc_10]
eval

,EBM model,f1,acc
0,full,0.739894,0.739870
1,reduced,0.735843,0.735818


Comparing the Area under the Curve for the full and reduced EBM models:

In [24]:
ebm_r_perf = ROC(ebm_r.predict_proba).explain_perf(X_test, y_test, name='EBM-R-Perf')
show([ebm_perf, ebm_r_perf])

<!-- http://127.0.0.1:7001/2613460591232/ -->
 Open in new window

It is extraordinary that restricting the EBM model from 34 to 10 features does not have any significant effect on model performance.

In [25]:
ebm_r_local = ebm_r.explain_local(X_test[:5], y_test[:5])
show(ebm_r_local)

<!-- http://127.0.0.1:7001/2613483930912/ -->